In [53]:
# Импортируем функцию urlopen и request из модуля request пакета urllib
from urllib.request import urlopen
from urllib.request import Request
# Импортируем модуль errors
from urllib.error import HTTPError
# Импортируем "прекрасный суп"
from bs4 import BeautifulSoup as bs
import re
import pandas as pd

In [28]:
# Поскольку сайт, который мы собираемся скрапить, защищен он ботов, необходимо доказать, что мы из мяса. Делаем это с помощью 
# объекта класса .Session
import requests as req
s = req.Session()
# Пишем HTTP-заголовки. Отправляются при каждом запросе. ЗАМЕНИТЬ НА СВОИ ПРИ НЕОБХОДИМОСТИ
heads = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36",
         "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"}


In [65]:
def get_me_some_info(url):
    global s, heads
    try: # Посылаем запрос с помощью Сессионс, притворяясь человеком
        html = session.get(url, headers=heads)
    except HTTPError as e:
        return "Ошибка HTTP. Проверь URL адрес, либо у тебя нет права доступа"
    # Если запрос удачный, работаем с "супом"
    try:
        HTML_Code = bs(html.text)
        result_holders = HTML_Code.findAll("div", {"class":"issue-item_metadata"})
        return result_holders
    except:
        return "Ай-нане-нане!"
    

In [106]:
# Напишем адрес, который будет запрашиваться при поиске. Адрес - результат выполнения функции
def search(KeyWords, page_size, earliest_date = None, latest_date = None):
    """
    KeyWords : list
        A list of keywords that you need to search for articles, e.g. ["mesophase","carbon"]
    
    page_size : int
        The size of page == the number of results shown in one page
        Possible values: 20, 50, 100
        
    earliest_date : int, optional
        The earliest date to search for articles from
        The format is as follows: 4 digits for year, 2 digits for month and two digits for day - "20210407"
        NOTE!!! If earliest_date is given, the argument "latest_date" must be given too
        
    latest_date : int, optional
        The date articles are searched to
        The format is as follows: 4 digits for year, 2 digits for month and two digits for day - "20210407"
    """
    search_page = "https://pubs.acs.org/action/doSearch?AllField="
    kw_position = 0
    while kw_position <= len(KeyWords)-1:
        search_page += KeyWords[kw_position] + "+"
        kw_position += 1
    search_page += "&startPage=0"
    if earliest_date != None and latest_date != None:
        search_page += "&Earliest=%5B" + str(earliest_date) + "+TO+" + str(latest_date) + "%5D"
    else:
        pass
    search_page += "&pageSize=" + str(page_size)
    return search_page

In [110]:
webpage = search(["gas","condensate"],100, 20200507, 20210107)

In [111]:
arts = get_me_some_info(webpage)

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pubs.acs.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\users\user\appdata\local\programs\python\python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pubs.acs.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [112]:

def extract_titles(info_holders):
    titles_list = list()
    for result in info_holders:
        if result.h2 != None:
            article_name = result.h2.get_text()
        else:
            article_name = "N/A"
        titles_list.append(article_name)
    return titles_list

def extract_journals(info_holders):
    journals_and_books = list()
    for result in info_holders:
        if result.find("span",{"class":"issue-item_chapter_bookTitle"}) != None:
            j_title = result.find("span",{"class":"issue-item_chapter_bookTitle"}).get_text() + ", " + "Book title"
        elif result.find("span",{"class":"issue-item_jour-name"}) != None:
            j_title = result.find("span",{"class":"issue-item_jour-name"}).get_text() + ", " + "Jour. title"
        else:
            j_title = "N/A"
        journals_and_books.append(j_title)
    return journals_and_books

def extract_authors(info_holders):
    authors = list()
    for result in info_holders:
        if result.ul != None:
            j_authors = result.ul.get_text()
        else:
            j_authors = "N/A"
        authors.append(j_authors)
    return authors

    
def extract_doi(info_holders):
    doi_list = list()
    for result in info_holders:
        if result.h2.a != None:
            doi = result.h2.a.get("href")
        else:
            doi = "N/A"
        doi_list.append(doi)
    return doi_list

def extract_dates(info_holders):
    

In [113]:
titles = extract_titles(arts)
journs = extract_journals(arts)
auths = extract_authors(arts)
dois = extract_doi(arts)

In [114]:
df = pd.DataFrame(columns = ["Article Head","Authors","Journal or book title","DOI"])


In [115]:
df["Article Head"] = titles
df["Authors"] = auths
df["Journal or book title"] = journs
df["DOI"] = dois

In [116]:
df

,Article Head,Authors,Journal or book title,DOI
0,Injection of Nanofluids with Fluorosurfactant-...,"Dahiana Galeano-Caro, Juan P. Villegas, Juan H...","Energy & Fuels, Jour. title",/doi/10.1021/acs.energyfuels.0c02689
1,Amphiphilic Thiol Polymer Nanogel Removes Envi...,"Yilin Zhang, Garret D. Bland, Jiajun Yan, Astr...","Environmental Science & Technology, Jour. title",/doi/10.1021/acs.est.0c05470
2,Potential of Light Oil and Condensates from De...,"Qiang Wang, Wanglu Jia*, Chiling Yu, Jianzhong...","Energy & Fuels, Jour. title",/doi/10.1021/acs.energyfuels.0c00553
3,Single-Protein Collapse Determines Phase Equil...,Han-Yi Chou and Aleksei Aksimentiev*,"The Journal of Physical Chemistry Letters, Jou...",/doi/10.1021/acs.jpclett.0c01222
4,Confined Growth and Controlled Coalescence/Sel...,"Dandan Xing, Rui Wang, Feifei Wu, and Xuefeng ...","ACS Applied Materials & Interfaces, Jour. title",/doi/10.1021/acsami.0c04922
...,...,...,...,...
95,First-Principles Algorithm for Air Quality Ele...,Bin Ouyang*,"ACS Sensors, Jour. title",/doi/10.1021/acssensors.0c01129
96,"Behavior of Isobaric Heat Capacity of 2,3,3,3-...",Noboru Kagawa* and Atsushi Matsuguchi,"Journal of Chemical & Engineering Data, Jour. ...",/doi/10.1021/acs.jced.0c00243
97,Outstanding Performance of Magnetically Separa...,"Chang Wang, Xufan Zhang, Jian Mei, Qixing Hu, ...","Environmental Science & Technology, Jour. title",/doi/10.1021/acs.est.0c01373
98,Modeling and Performance Analysis of Shale Oil...,"Hongwei Li, Guixian Li, Qingchun Yang, and Hua...","Energy & Fuels, Jour. title",/doi/10.1021/acs.energyfuels.0c01709
